In [45]:
import random
from enum import Enum

In [1]:
import torch
from transformers import BertModel, BertTokenizer
from scipy.spatial.distance import cosine

In [3]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/Users/martincastro/Library/Python/3.8/lib/python/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Defining Actions that can be taken by an AI agent

In [38]:
class Actions(Enum):
    RESPOND_ANGRILY = "respond_angrily"
    RESPOND_CALMLY = "respond_calmly"
    RESPOND_HAPPILY = "respond_happily"
    RESPOND_SADLY = "respond_sadly"
    TELL_JOKE_GENERAL = "tell_joke_general"
    TELL_JOKE_KNOCK_KNOCK = "tell_joke_knock_knock"
    TELL_JOKE_PUN = "tell_joke_pun"
    LAUGH_LOUDLY = "laugh_loudly"
    LAUGH_SOFTLY = "laugh_softly"
    LAUGH_GIGGLY = "laugh_giggly"
    ASK_QUESTION_GENERAL = "ask_question_general"
    ASK_QUESTION_PERSONAL = "ask_question_personal"
    ASK_QUESTION_TECHNICAL = "ask_question_technical"
    SING_SONG_HAPPY = "sing_song_happy"
    SING_SONG_SAD = "sing_song_sad"
    SING_SONG_POP = "sing_song_pop"
    PLAY_MUSIC_CLASSICAL = "play_music_classical"
    PLAY_MUSIC_ROCK = "play_music_rock"
    PLAY_MUSIC_JAZZ = "play_music_jazz"
    DANCE_SALSA = "dance_salsa"
    DANCE_HIPHOP = "dance_hiphop"
    DANCE_BALLROOM = "dance_ballroom"
    GREET_VIEWERS_CASUALLY = "greet_viewers_casually"
    GREET_VIEWERS_FORMALLY = "greet_viewers_formally"
    GREET_VIEWERS_ENERGETICALLY = "greet_viewers_energetically"
    READ_COMMENT_POSITIVE = "read_comment_positive"
    READ_COMMENT_NEGATIVE = "read_comment_negative"
    READ_COMMENT_QUESTION = "read_comment_question"
    THANK_VIEWERS_GRATEFULLY = "thank_viewers_gratefully"
    THANK_VIEWERS_BRIEFLY = "thank_viewers_briefly"
    THANK_VIEWERS_HEARTILY = "thank_viewers_heartily"
    ENCOURAGE_INTERACTION_DIRECTLY = "encourage_interaction_directly"
    ENCOURAGE_INTERACTION_SUBTLY = "encourage_interaction_subtly"
    ENCOURAGE_INTERACTION_FUNLY = "encourage_interaction_funly"
    SWITCH_SCENE_TO_MAIN = "switch_scene_to_main"
    SWITCH_SCENE_TO_BE_RIGHT_BACK = "switch_scene_to_be_right_back"
    SWITCH_SCENE_TO_ENDING = "switch_scene_to_ending"
    ADJUST_LIGHTING_BRIGHTER = "adjust_lighting_brighter"
    ADJUST_LIGHTING_DIMMER = "adjust_lighting_dimmer"
    ADJUST_LIGHTING_COLORFUL = "adjust_lighting_colorful"
    DISPLAY_TEXT_MESSAGE = "display_text_message"
    DISPLAY_TEXT_WARNING = "display_text_warning"
    DISPLAY_TEXT_UPDATE = "display_text_update"
    DISPLAY_POLL_VIEWER_CHOICE = "display_poll_viewer_choice"
    DISPLAY_POLL_FEEDBACK = "display_poll_feedback"
    DISPLAY_POLL_OPINIONS = "display_poll_opinions"
    ASK_FOR_FEEDBACK_GENERAL = "ask_for_feedback_general"
    ASK_FOR_FEEDBACK_SPECIFIC = "ask_for_feedback_specific"
    GIVE_SHOUTOUT_PERSONAL = "give_shoutout_personal"
    GIVE_SHOUTOUT_SPONSOR = "give_shoutout_sponsor"
    SHARE_STORY_PERSONAL = "share_story_personal"
    SHARE_STORY_EDUCATIONAL = "share_story_educational"
    SHARE_STORY_FUNNY = "share_story_funny"

### Normalizing the actions so that they can be tokenized

In [39]:
actions = [
    action.value.lower().replace("_", " ")
    for action in Actions
]
print(actions)

['respond angrily', 'respond calmly', 'respond happily', 'respond sadly', 'tell joke general', 'tell joke knock knock', 'tell joke pun', 'laugh loudly', 'laugh softly', 'laugh giggly', 'ask question general', 'ask question personal', 'ask question technical', 'sing song happy', 'sing song sad', 'sing song pop', 'play music classical', 'play music rock', 'play music jazz', 'dance salsa', 'dance hiphop', 'dance ballroom', 'greet viewers casually', 'greet viewers formally', 'greet viewers energetically', 'read comment positive', 'read comment negative', 'read comment question', 'thank viewers gratefully', 'thank viewers briefly', 'thank viewers heartily', 'encourage interaction directly', 'encourage interaction subtly', 'encourage interaction funly', 'switch scene to main', 'switch scene to be right back', 'switch scene to ending', 'adjust lighting brighter', 'adjust lighting dimmer', 'adjust lighting colorful', 'display text message', 'display text warning', 'display text update', 'displ

### Tokenizing actions so that they can be converted into embeddings

In [40]:
tokens = [
    tokenizer(action, padding=True, return_tensors="pt")
    for action in actions
]
print("Tokenized Actions:", len(tokens))

Tokenized Actions: 53


### Calculating the embeddings of the tokens of the actions

In [41]:
with torch.no_grad():
    embeddings = [
        model(**row).last_hidden_state[:, 1, :].squeeze(0)
        for row in tokens
    ]

print(embeddings[0].shape)

torch.Size([768])


### Calculating the similarity of the embeddings of the actions

In [67]:
def sample(sample_size=10) -> None:
    i = random.choice(range(len(actions)))
    similarities = {
        j: 1 - cosine(embeddings[i].numpy(), embeddings[j].numpy())
        for j in range(len(actions))
    }
    sorted_similarities = sorted(similarities.items(), key=lambda item: item[1], reverse=True)
    for k in range(sample_size):
        j, similarity = sorted_similarities[k]
        print(f"The cosine similarity between '{actions[i]}' and '{actions[j]}' is: {similarity:.4f}")

In [68]:
sample()

The cosine similarity between 'display text warning' and 'display text warning' is: 1.0000
The cosine similarity between 'display text warning' and 'display text update' is: 0.8900
The cosine similarity between 'display text warning' and 'display text message' is: 0.8815
The cosine similarity between 'display text warning' and 'display poll feedback' is: 0.7896
The cosine similarity between 'display text warning' and 'display poll viewer choice' is: 0.6804
The cosine similarity between 'display text warning' and 'display poll opinions' is: 0.6689
The cosine similarity between 'display text warning' and 'read comment negative' is: 0.5958
The cosine similarity between 'display text warning' and 'ask question personal' is: 0.5830
The cosine similarity between 'display text warning' and 'ask question technical' is: 0.5689
The cosine similarity between 'display text warning' and 'ask question general' is: 0.5586


In [69]:
sample()

The cosine similarity between 'adjust lighting colorful' and 'adjust lighting colorful' is: 1.0000
The cosine similarity between 'adjust lighting colorful' and 'adjust lighting brighter' is: 0.8317
The cosine similarity between 'adjust lighting colorful' and 'adjust lighting dimmer' is: 0.7520
The cosine similarity between 'adjust lighting colorful' and 'encourage interaction funly' is: 0.7317
The cosine similarity between 'adjust lighting colorful' and 'ask for feedback general' is: 0.7262
The cosine similarity between 'adjust lighting colorful' and 'ask for feedback specific' is: 0.7057
The cosine similarity between 'adjust lighting colorful' and 'play music rock' is: 0.6825
The cosine similarity between 'adjust lighting colorful' and 'respond calmly' is: 0.6623
The cosine similarity between 'adjust lighting colorful' and 'display poll opinions' is: 0.6550
The cosine similarity between 'adjust lighting colorful' and 'encourage interaction subtly' is: 0.6528


In [71]:
sample()

The cosine similarity between 'play music jazz' and 'play music jazz' is: 1.0000
The cosine similarity between 'play music jazz' and 'play music classical' is: 0.9122
The cosine similarity between 'play music jazz' and 'play music rock' is: 0.8627
The cosine similarity between 'play music jazz' and 'dance salsa' is: 0.5921
The cosine similarity between 'play music jazz' and 'ask question personal' is: 0.5792
The cosine similarity between 'play music jazz' and 'ask question technical' is: 0.5774
The cosine similarity between 'play music jazz' and 'ask question general' is: 0.5698
The cosine similarity between 'play music jazz' and 'display poll opinions' is: 0.5658
The cosine similarity between 'play music jazz' and 'sing song pop' is: 0.5652
The cosine similarity between 'play music jazz' and 'encourage interaction subtly' is: 0.5636


In [73]:
sample()

The cosine similarity between 'respond angrily' and 'respond angrily' is: 1.0000
The cosine similarity between 'respond angrily' and 'respond happily' is: 0.8764
The cosine similarity between 'respond angrily' and 'respond sadly' is: 0.8546
The cosine similarity between 'respond angrily' and 'respond calmly' is: 0.8068
The cosine similarity between 'respond angrily' and 'ask question general' is: 0.6029
The cosine similarity between 'respond angrily' and 'ask question personal' is: 0.6008
The cosine similarity between 'respond angrily' and 'ask question technical' is: 0.5974
The cosine similarity between 'respond angrily' and 'read comment negative' is: 0.5601
The cosine similarity between 'respond angrily' and 'greet viewers formally' is: 0.5338
The cosine similarity between 'respond angrily' and 'read comment question' is: 0.5326
